In [1]:
from glob import glob
import IPython.display as ipd
import matplotlib.pyplot as plt
from pyAudioAnalysis import audioBasicIO as aIo
from pyAudioAnalysis import ShortTermFeatures as sf
import librosa
import librosa.display
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
file_paths = glob('../Downloads/Raw/Video/Segmented/*')

In [3]:
data = pd.read_csv('../Downloads/Raw/OpinionLevelSentiment.csv', names = ['Seg Start', 'Segment end', 'Name','Seg No', 'Review'])

In [4]:
dataframe = pd.DataFrame(data)
dataframe

,Seg Start,Segment end,Name,Seg No,Review
0,27.879138,32.927664,_dI--eQ6qVU,1,2.4
1,45.548980,47.783900,_dI--eQ6qVU,2,3.0
2,47.783900,59.247846,_dI--eQ6qVU,3,-0.2
3,59.247846,68.277324,_dI--eQ6qVU,4,3.0
4,68.277324,71.200680,_dI--eQ6qVU,5,3.0
...,...,...,...,...,...
2194,147.217914,153.703175,ZUXBRvtny7o,30,0.4
2195,157.265079,161.256009,ZUXBRvtny7o,31,1.4
2196,161.256009,173.049206,ZUXBRvtny7o,32,0.4
2197,173.049206,175.224263,ZUXBRvtny7o,33,0.8


In [5]:
dataframe['Polarity'] = dataframe['Review']
for i in range(len(dataframe)):
    if(dataframe['Review'][i]>0):
        #positive
        dataframe['Polarity'][i] = 1
    else:
        #negative
        dataframe['Polarity'][i] = 0

In [6]:
dataframe

,Seg Start,Segment end,Name,Seg No,Review,Polarity
0,27.879138,32.927664,_dI--eQ6qVU,1,2.4,1.0
1,45.548980,47.783900,_dI--eQ6qVU,2,3.0,1.0
2,47.783900,59.247846,_dI--eQ6qVU,3,-0.2,0.0
3,59.247846,68.277324,_dI--eQ6qVU,4,3.0,1.0
4,68.277324,71.200680,_dI--eQ6qVU,5,3.0,1.0
...,...,...,...,...,...,...
2194,147.217914,153.703175,ZUXBRvtny7o,30,0.4,1.0
2195,157.265079,161.256009,ZUXBRvtny7o,31,1.4,1.0
2196,161.256009,173.049206,ZUXBRvtny7o,32,0.4,1.0
2197,173.049206,175.224263,ZUXBRvtny7o,33,0.8,1.0


In [7]:
p = 0
n = 0
for i in range(2199):
    if(dataframe['Polarity'][i] == 1):
        p+=1
    else:
        n+=1

In [8]:
p

1080

In [9]:
n

1119

In [10]:
def load_file(filepath):
    y,sr = librosa.load(filepath)
    return (y,sr)

In [11]:
def divide_segments(y_time,start):
    # divide into 50 ms segments
    # 50ms -> means 1102 samples    
    y_segmented = y_time[start:start+1102]
    return y_segmented

In [12]:
def stft(y_seg , sr, window, step):
    [f,f_name] = sf.feature_extraction(y_seg,sr, window, step,  deltas = False)
    return f

In [13]:
def feature_matricization(y_time):
    y_seg = divide_segments(y_time,0)
    y_stft = stft(y_seg,sr,0.05*sr,0.05*sr)
    y_stft = y_stft.reshape(34)
    
    start = 1111
    y_stfts = y_stft
    while(start <= len(y_time)-1111):
        y_seg = divide_segments(y_time,start)
        #print(y_seg)
        start += 1111
        y_stft = stft(y_seg,sr,0.05*sr,0.05*sr)
        #y_stft_arr = np.asarray(y_stft).T
        y_stft = y_stft.reshape(34)
        y_stfts = np.vstack((y_stfts, y_stft))
        #print(y_stft_arr.shape)

        #np.column_stack((y_stfts, y_stft_arr))
        
    if(len(y_stfts)<300):
        #cal how less
        x = 300 - len(y_stfts)%300
        zeros_arr = np.zeros(shape = (x,34))
        x_arr = np.asanyarray(y_stfts)
        x_arr = x_arr.reshape(-1,34)
        x_final = np.vstack((x_arr,zeros_arr))
        
        return x_final
    if(len(y_stfts)>=300):
        y_stfts = y_stfts[0:300]
        return np.asanyarray(y_stfts)

In [14]:
file_path = '../Downloads/Raw/Video/Segmented/' + dataframe['Name'][0] +  '_' + dataframe['Seg No'][0].__str__() +'.mp4'
label = dataframe['Polarity'][0]
labels = np.asarray(label)
y,sr = librosa.load(file_path)
y_time = librosa.frames_to_time(y)
feat = feature_matricization(y_time)
features = feat
for i in range(len(file_paths)):
    if(i!=0):
        #print(i)
        file_path = '../Downloads/Raw/Video/Segmented/' + dataframe['Name'][i] +  '_' + dataframe['Seg No'][i].__str__() +'.mp4'
        label = dataframe['Polarity'][i]
        labels = np.vstack((labels,label))
        y,sr = librosa.load(file_path)
        y_time = librosa.frames_to_time(y)
        feat = feature_matricization(y_time)
        features = np.vstack((features,feat))

In [15]:
labels.shape

(2199, 1)

In [16]:
features.shape

(659700, 34)

In [17]:
# divide features into 300 * 34 segements

In [27]:
# no of segments
indices = features.shape[0]/300
indices

2199.0

In [28]:
features_segments = np.split(features,indices)

In [29]:
feat_seg_arr = np.asanyarray(features_segments)

In [ ]:
feat_seg_arr.shape

In [ ]:
labels = labels.astype('float32')

In [ ]:
feat_seg_arr

In [ ]:
feat_seg_arr[1][1]

In [ ]:
feat_seg_arr_norm = feat_seg_arr
for i in range(feat_seg_arr.shape[0]):
    for j in range(feat_seg_arr.shape[1]):
        for k in range(feat_seg_arr.shape[2]):
            if(feat_seg_arr[i][j].max()!=0):
                
                feat_seg_arr_norm[i][j][k] = feat_seg_arr[i][j][k]/feat_seg_arr.max()

In [ ]:
feat_seg_arr_norm

In [ ]:
# train test split

In [30]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(feat_seg_arr, labels)

In [34]:
print(X_train[10] == X_train[1])

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]


In [35]:
X_train.shape

(1649, 300, 34)

In [36]:
y_train.shape

(1649, 1)

In [37]:
import tensorflow as tf

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout

In [39]:
model = Sequential()

2022-07-17 16:14:33.908259: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [44]:
model = Sequential()
model.add(layers.Conv1D(256, 5,padding='same',
                 input_shape=(300,34)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(128, 5,padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling1D(pool_size=(4)))
model.add(layers.Dropout(0.1))
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

In [45]:
model.compile(optimizer = 'Adam' , loss = tf.keras.losses.binary_crossentropy, metrics = ['accuracy'])

In [46]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 300, 256)          43776     
                                                                 
 activation_3 (Activation)   (None, 300, 256)          0         
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 37, 256)          0         
 1D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 37, 256)           0         
                                                                 
 conv1d_3 (Conv1D)           (None, 37, 128)           163968    
                                                                 
 activation_4 (Activation)   (None, 37, 128)           0         
                                                      

In [47]:
model.fit(X_train,y_train,epochs=20)

Epoch 1/20
52/52 [==============================] - 4s 60ms/step - loss: 0.9571 - accuracy: 0.5021
Epoch 2/20
52/52 [==============================] - 3s 54ms/step - loss: 0.6895 - accuracy: 0.5403
Epoch 3/20
52/52 [==============================] - 3s 50ms/step - loss: 0.6900 - accuracy: 0.5355
Epoch 4/20
52/52 [==============================] - 3s 50ms/step - loss: 0.6870 - accuracy: 0.5476
Epoch 5/20
52/52 [==============================] - 3s 55ms/step - loss: 0.6796 - accuracy: 0.5476
Epoch 6/20
52/52 [==============================] - 3s 54ms/step - loss: 0.6891 - accuracy: 0.5500
Epoch 7/20
52/52 [==============================] - 3s 54ms/step - loss: 0.6832 - accuracy: 0.5403
Epoch 8/20
52/52 [==============================] - 3s 57ms/step - loss: 0.6785 - accuracy: 0.5567
Epoch 9/20
52/52 [==============================] - 3s 58ms/step - loss: 0.6901 - accuracy: 0.5531
Epoch 10/20
52/52 [==============================] - 3s 59ms/step - loss: 0.6728 - accuracy: 0.5719
Epoch 11/

In [48]:
from sklearn.metrics import accuracy_score

In [49]:
p = model.predict(X_test)

In [50]:
for i in range(len(p)):
    if p[i]>0.65:
        p[i] = 1.0
    else:
        p[i] = 0.0
p

array([[0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],

In [51]:
y_test

array([[1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],

In [52]:
p_train = model.predict(X_train)

In [53]:
p_train

array([[0.2725746 ],
       [0.6199038 ],
       [0.49650693],
       ...,
       [0.8584292 ],
       [0.4920748 ],
       [0.71316075]], dtype=float32)

In [54]:
y_train

array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])

In [55]:
from sklearn.metrics import accuracy_score

In [56]:
accuracy = accuracy_score(y_test,p)

In [57]:
accuracy

0.49636363636363634